In [1]:
import pandas as pd
import random
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

%config InlineBackend.figure_format = 'retina'

import math
import seaborn as sns
sns.set_context('talk')

## Training Images and Metadata

In [2]:
train_metadata = pd.read_csv('carnet_dataset/train/metadata.csv')
train_metadata.head()

,car,bus,truck,train,motorcycle,bicycle,airplane,boat,traffic light,stop sign,vehicle,signal,file_name
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,000000000064.jpg
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,000000000073.jpg
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,000000000074.jpg
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,000000000081.jpg
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,000000000086.jpg


## Score Images and Metadata

In [3]:
# Score metadata defines the expected order of the photos in the submitted predictions file.

score_metadata = pd.read_csv('carnet_dataset/score/metadata.csv')
score_metadata.head()

,file_name
0,000000000071.jpg
1,000000000149.jpg
2,000000000260.jpg
3,000000000307.jpg
4,000000000690.jpg


## Creating Tensorflow Dataset (Single Image, Labeled Segment)

In [4]:
# This configures the GPU to be used by Tensorflow.
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

2022-10-31 05:36:47.059623: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 05:36:47.112673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 05:36:47.113531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [5]:
### Splitting the train data into train, validate, and test 
from sklearn.model_selection import train_test_split
train_metadata, test_metadata = train_test_split(train_metadata, test_size=0.20, random_state=42)
train_metadata, val_metadata = train_test_split(train_metadata, test_size=0.25, random_state=42)

In [6]:
# See Python Generator
# https://peps.python.org/pep-0255/
def build_generator_labeled_single(metadata: pd.DataFrame):
    def generator():
        for _, row in metadata.iterrows():
            
            training_path = 'carnet_dataset/train/images/' + row['file_name']
            training_np = np.array(Image.open(training_path)).astype(np.float32)
            
            model_input = training_np
            model_output_1 = (row["signal"], )
            # model_output_2 = (row["vehicle"], )
            yield (model_input, (model_output_1))
            
    return generator


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_labeled_single(metadata: pd.DataFrame) -> tf.data.Dataset:
    model_input = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore
    model_output = tf.TensorSpec(shape=(1, ), dtype=tf.float32)  # type: ignore

    dataset_signature = (model_input, (model_output))

    dataset = tf.data.Dataset.from_generator(
        build_generator_labeled_single(metadata), 
        output_signature=dataset_signature
    )

    return dataset


In [7]:
# See Python Generator
# https://peps.python.org/pep-0255/
def build_generator_labeled_single_vehicle(metadata: pd.DataFrame):
    def generator():
        for _, row in metadata.iterrows():
            
            training_path = 'carnet_dataset/train/images/' + row['file_name']
            training_np = np.array(Image.open(training_path)).astype(np.float32)
            
            model_input = training_np
            # model_output_1 = (row["signal"], )
            model_output_2 = (row["vehicle"], )
            yield (model_input, (model_output_2))
            
    return generator


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_labeled_single_vehicle(metadata: pd.DataFrame) -> tf.data.Dataset:
    model_input = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore
    model_output = tf.TensorSpec(shape=(1, ), dtype=tf.float32)  # type: ignore

    dataset_signature = (model_input, (model_output))

    dataset = tf.data.Dataset.from_generator(
        build_generator_labeled_single_vehicle(metadata), 
        output_signature=dataset_signature
    )

    return dataset


In [8]:
batch_size = 128

In [9]:
train_single_dataset = build_dataset_labeled_single(train_metadata).batch(batch_size)
train_single_dataset_vehicle = build_dataset_labeled_single_vehicle(train_metadata).batch(batch_size)
# model_input, (model_output_1, model_output_2) = next(iter(train_single_dataset))
# display(model_input.shape, model_output_1.shape, model_output_2.shape)

2022-10-31 05:37:06.145756: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 05:37:06.146391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 05:37:06.147302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 05:37:06.148127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [10]:
val_single_dataset = build_dataset_labeled_single(val_metadata).batch(batch_size)
val_single_dataset_vehicle = build_dataset_labeled_single_vehicle(val_metadata).batch(batch_size)
# model_input, (model_output_1, model_output_2) = next(iter(val_single_dataset))
# display(model_input.shape, model_output_1.shape, model_output_2.shape)

In [11]:
test_single_dataset = build_dataset_labeled_single(test_metadata).batch(batch_size)
test_single_dataset_vehicle = build_dataset_labeled_single_vehicle(test_metadata).batch(batch_size)
# model_input, (model_output_1, model_output_2) = next(iter(test_single_dataset))
# display(model_input.shape, model_output_1.shape, model_output_2.shape)

## Creating Tensorflow Dataset (Single Image, Score Segment)

In [12]:
import tensorflow as tf
import numpy as np

# See Python Generator
# https://peps.python.org/pep-0255/

def generator_score_single():
    metadata = pd.read_csv('carnet_dataset/score/metadata.csv')
    
    for _, row in metadata.iterrows():
        image_path = 'carnet_dataset/score/images/' + row['file_name']

        image_np = np.array(Image.open(image_path)).astype(np.float32)

        model_input = image_np
        yield model_input


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_score_single() -> tf.data.Dataset:
    model_input = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore

    dataset_signature = model_input

    dataset = tf.data.Dataset.from_generator(
        generator_score_single, 
        output_signature=dataset_signature
    )

    return dataset

In [13]:
score_single_dataset = build_dataset_score_single().batch(batch_size)
# model_input = next(iter(score_single_dataset))
# display(model_input.shape)

# Model

In [43]:
### Getting pre-trained MobileNet Architecture, 
### using global average pooling at the end and excluding the top dense layer
# https://arxiv.org/pdf/1801.04381.pdf
# https://keras.io/api/applications/mobilenet/#mobilenetv2-function
eff_net = tf.keras.applications.EfficientNetV2B1(
    include_top=False,
    input_shape=(224, 224, 3),
    pooling="avg",
    weights="imagenet"
)

In [44]:
### Freezing the parameters of the MobileNet layers
eff_net.trainable = False

In [45]:
# See Keras Functional API
# https://keras.io/guides/functional_api/

### Three inputs: real image, generated image, caption embeddings
inp = tf.keras.layers.Input((224, 224, 3))

rescale = tf.keras.layers.Rescaling(1./255)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation((-1, 1)),
    tf.keras.layers.RandomContrast((0, 1)),
    tf.keras.layers.RandomHeight((-0.5, 0.5)),
    tf.keras.layers.RandomWidth((-0.5, 0.5))
    
])

img = tf.keras.models.Sequential([
    rescale,
    data_augmentation,
    eff_net,
])

### Using MobileNet for the images and Dense layer architecture for the caption embeddings
outputs = eff_net(inp)

### Concatenating the three outputs and using Dence layers on top, with dropouts to avoid overfitting
outputs_1 = tf.keras.layers.Dense(256, activation='relu')(outputs)
# outputs_1 = tf.keras.layers.Dropout(rate = 0.2)(outputs_1)
outputs_1 = tf.keras.layers.Dense(128, activation='relu')(outputs_1)
# outputs_1 = tf.keras.layers.Dropout(rate = 0.25)(outputs_1)
outputs_1 = tf.keras.layers.Dense(64, activation='relu')(outputs_1)
# outputs_1 = tf.keras.layers.Dropout(rate = 0.25)(outputs_2)
# outputs_1 = tf.keras.layers.Dense(32, activation='relu')(outputs_1)
outputs_1 = tf.keras.layers.Dense(1)(outputs_1)

# outputs_1 = tf.keras.layers.Dense(256, activation='relu')(outputs)
# outputs_1 = tf.keras.layers.Dropout(rate = 0.2)(outputs_1)
# outputs_1 = tf.keras.layers.Dense(128, activation='relu')(outputs_1)
# outputs_1 = tf.keras.layers.Dropout(rate = 0.25)(outputs_1)
# outputs_1 = tf.keras.layers.Dense(64, activation='relu')(outputs)
# outputs_1 = tf.keras.layers.Dropout(rate = 0.25)(outputs_1)
# outputs_2 = tf.keras.layers.Dense(32, activation='relu')(outputs)
outputs_2 = tf.keras.layers.Dense(16, activation='relu')(outputs)
outputs_2 = tf.keras.layers.Dense(8, activation='relu')(outputs_2)
outputs_2 = tf.keras.layers.Dense(4, activation='relu')(outputs_2)
outputs_2 = tf.keras.layers.Dense(2, activation='relu')(outputs_2)
outputs_2 = tf.keras.layers.Dense(1)(outputs_2)

model1 = tf.keras.models.Model(inputs=inp, outputs=[outputs_1])
model2 = tf.keras.models.Model(inputs=inp, outputs=[outputs_2])

In [46]:
### Adam optimizer with 0.01 learning rate
model1.compile(
    optimizer=tf.keras.optimizers.Adam(
        lr=0.001
    ),
    loss='mean_squared_error'
)

model2.compile(
    optimizer=tf.keras.optimizers.Adam(
        lr=0.001
    ),
    loss='mean_squared_error'
)

/home/ec2-user/SageMaker/setup/miniconda/envs/ucla_deeplearning/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [47]:
### Checkpoint path 
checkpoint_path = "weights/car_27_d1"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [48]:
import numpy as np
import tensorflow as tf
np.random.seed(11)
tf.random.set_seed(11)

### Callback to save model log / results after each epoch
csv_logger = tf.keras.callbacks.CSVLogger("model27_d1_history_log.csv", append=True)

### Callback to save weights after every epoch i.e., checkpoint
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_best_only=True, monitor = 'val_loss') 

### Callback to stop the training if there's no improvement in validation loss for 3 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model1.fit(
    train_single_dataset,
    epochs=128,
    shuffle=True,
    validation_data=(val_single_dataset),
    callbacks=[stop_early, cp_callback, csv_logger]
)

Epoch 1/128
    109/Unknown - 78s 653ms/step - loss: 1.4388
Epoch 1: val_loss improved from inf to 1.21313, saving model to weights/car_27_d1
109/109 [==============================] - 104s 893ms/step - loss: 1.4388 - val_loss: 1.2131
Epoch 2/128
109/109 [==============================] - ETA: 0s - loss: 1.1862
Epoch 2: val_loss improved from 1.21313 to 1.19553, saving model to weights/car_27_d1
109/109 [==============================] - 96s 878ms/step - loss: 1.1862 - val_loss: 1.1955
Epoch 3/128
109/109 [==============================] - ETA: 0s - loss: 0.9907
Epoch 3: val_loss did not improve from 1.19553
109/109 [==============================] - 95s 875ms/step - loss: 0.9907 - val_loss: 1.2291
Epoch 4/128
109/109 [==============================] - ETA: 0s - loss: 0.7860
Epoch 4: val_loss did not improve from 1.19553
109/109 [==============================] - 95s 874ms/step - loss: 0.7860 - val_loss: 1.2637
Epoch 5/128
109/109 [==============================] - ETA: 0s - loss: 0.63

In [49]:
checkpoint_path = "weights/car_27_d2"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [50]:
import numpy as np
import tensorflow as tf
np.random.seed(11)
tf.random.set_seed(11)

### Callback to save model log / results after each epoch
csv_logger = tf.keras.callbacks.CSVLogger("model27_d2_history_log.csv", append=True)

### Callback to save weights after every epoch i.e., checkpoint
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_best_only=True, monitor = 'val_loss') 

### Callback to stop the training if there's no improvement in validation loss for 3 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model2.fit(
    train_single_dataset_vehicle,
    epochs=128,
    shuffle=True,
    validation_data=(val_single_dataset_vehicle),
    callbacks=[stop_early, cp_callback, csv_logger]
)

Epoch 1/128
    109/Unknown - 79s 656ms/step - loss: 12.3840
Epoch 1: val_loss improved from inf to 9.08405, saving model to weights/car_27_d2
109/109 [==============================] - 105s 896ms/step - loss: 12.3840 - val_loss: 9.0840
Epoch 2/128
109/109 [==============================] - ETA: 0s - loss: 9.0927
Epoch 2: val_loss improved from 9.08405 to 8.15117, saving model to weights/car_27_d2
109/109 [==============================] - 96s 880ms/step - loss: 9.0927 - val_loss: 8.1512
Epoch 3/128
109/109 [==============================] - ETA: 0s - loss: 8.2008
Epoch 3: val_loss improved from 8.15117 to 7.80022, saving model to weights/car_27_d2
109/109 [==============================] - 95s 876ms/step - loss: 8.2008 - val_loss: 7.8002
Epoch 4/128
109/109 [==============================] - ETA: 0s - loss: 7.6604
Epoch 4: val_loss improved from 7.80022 to 7.65554, saving model to weights/car_27_d2
109/109 [==============================] - 96s 879ms/step - loss: 7.6604 - val_loss: 7.

In [ ]:
checkpoint_path = "weights/car_27_d1"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
### Loading the weights from previous checkpoint
model1.load_weights(checkpoint_path)

In [ ]:
checkpoint_path = "weights/car_27_d2"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
### Loading the weights from previous checkpoint
model2.load_weights(checkpoint_path)

In [55]:
val_pred1 = model1.predict(val_single_dataset)
val_pred2 = model2.predict(val_single_dataset_vehicle)

In [56]:
val_signal = []
val_vehicle = []
for i in range(len(val_pred1)):
    if val_pred1[i][0]>0:
        val_signal.append(val_pred1[i][0])
    else:
        val_signal.append(0)
    if val_pred2[i][0]>0:
        val_vehicle.append(val_pred2[i][0])
    else:
        val_vehicle.append(0)

In [57]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(val_metadata["signal"].values, val_pred1))
print(mean_squared_error(val_metadata["vehicle"].values, val_pred2))

1.1955249283399287
7.576044090440667


In [58]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(val_metadata["signal"].values, val_signal))
print(mean_squared_error(val_metadata["vehicle"].values, val_vehicle))

1.1858136388895872
7.576044090440667


In [59]:
### Evaluating the model on validation set
model1.evaluate(val_single_dataset, verbose = 1)
model2.evaluate(val_single_dataset_vehicle, verbose = 1)

37/37 [==============================] - 24s 642ms/step - loss: 7.5760


7.576045036315918

In [60]:
### Evaluating the model on test set
model1.evaluate(test_single_dataset, verbose = 1)
model2.evaluate(test_single_dataset_vehicle, verbose = 1)

37/37 [==============================] - 24s 646ms/step - loss: 7.7487


7.748720169067383

In [ ]:
### Predicting the probabilities on score set
score_y_hat1 = model1.predict(score_single_dataset, verbose = 1)
score_y_hat2 = model2.predict(score_single_dataset, verbose = 1)

46/46 [==============================] - 31s 640ms/step
     42/Unknown - 29s 657ms/step

In [ ]:
signal = []
vehicle = []
for i in range(len(score_y_hat1)):
    if score_y_hat1[i][0]>0:
        signal.append(score_y_hat1[i][0])
    else:
        signal.append(0)
    if score_y_hat2[i][0]>0:
        vehicle.append(score_y_hat2[i][0])
    else:
        vehicle.append(0)

# Saving the model and predictions

In [40]:
# Save your predictions on the Score segment as a Pandas data frame into a variable named `score_y_hat`.
# The data frame should contain 2 columns: signal and vehicle.
# The following is an EXAMPLE of what this data frame could be.
score_y_hat = pd.DataFrame({
    'signal': signal,
    'vehicle':vehicle
})

# Use the following asserts to check the type and shape of the final predictions.
assert type(score_y_hat) == pd.DataFrame
assert score_y_hat.shape == (score_metadata.shape[0], 2)
assert (score_y_hat.columns == ['signal', 'vehicle']).all()

In [41]:
# Use the following code to save the final predictions.
import os 
model_dir = 'carnet_model'
os.makedirs(model_dir, exist_ok=True)
score_y_hat.to_parquet(f'{model_dir}/score_y_hat.parquet')

In [42]:
import os 
import pandas as pd

# os.makedirs(model_dir, exist_ok=True)

# Next, let's save the model's definition.
import json
with open(f'{model_dir}/keras_signal_model.json', 'w') as f:
    f.write(json.dumps(json.loads(model1.to_json()), indent=True))
    
with open(f'{model_dir}/keras_vehicle_model.json', 'w') as f:
    f.write(json.dumps(json.loads(model2.to_json()), indent=True))

# Finally, let's save the learned parameters.
model1.save_weights(f'{model_dir}/keras_parameters_signal_model.h5')
model2.save_weights(f'{model_dir}/keras_parameters_vehicle_model.h5')
# You now have the following files to be uploaded to Moodle:
# 1. This notebook and any other Python code you used to train the final model.
# 2. keras_model.json -- the model's definition
# 3. keras_parameters.json -- the model's trained parameters
# 4. score_y_hat.parquet - the model's output on the score dataset

In [28]:
# Next, save each model's defition (JSON file) and parameters (H5 file).

# You now have the following files to be submit:
# 1. This notebook and any other Python code you used to train the final model.
# 2. definitions of all trained models
# 3. parameters of all trained models
# 4. score_y_hat.parquet - the model's output on the score segment